In [1]:
#load all necessary libraries
import pandas as pd 
import numpy as np 
import scipy as scp
import sklearn
import statsmodels.api as sm
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix


In [ ]:
#read the dataset
abalone_df = pd.read_csv('abalone.csv') 
abalone_df.head()

There are 10 variables of which the first - SEX - will be used as the dependent variable.   
 Additional Information

Predicting the age of abalone from physical measurements.  
The age of abalone is determined by cutting the shell through the cone, staining it, 
and counting the number of rings through a microscope -- a boring and time-consuming task.  
Other measurements, which are easier to obtain, are used to predict the age.  
Further information, such as weather patterns and location 
(hence food availability) may be required to solve the problem.

In [ ]:
Abalone_sex=abalone_df['Sex'].value_counts()

print(Abalone_sex)


Description Of Data: 
The data is sourced from study of Abalone in Tasmania. 
It can be found at the UCI Machine Learning Repository. The dataset contains 4,177 observations and 9 variables. 

SEX = M (male), F (female), I (infant) 

LENGTH = Longest shell length in mm 

DIAM = Diameter perpendicular to length in mm 

HEIGHT = Height perpendicular to length and diameter in mm 

WHOLE = Whole weight of abalone in grams 

SHUCK = Shucked weight of meat in grams 

VISCERA = Viscera weight in grams 

SHELL = Shell weight after drying in grams 

RINGS = Age (+1.5 gives the age in years) 



We are now ready to partition the dataset:

In [ ]:
#Create training and test datasets
#CLASS was recoded into SIZE_CLASS to change from string to integer
#CLASS needs to be dropped
X = abalone_df.drop(['Sex'], axis=1) 
y = abalone_df['Sex']

print(list(X.columns.values)) 

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty=None, solver='newton-cg').fit(X_train, y_train)

preds = model1.predict(X_test)

params = model1.get_params()
print(params)

Intercept And Coefficients: 
The intercept and coefficients are stored in model1.intercept and model1. 
coef_ respectively. Here we need to spend a bit of time, 
because the output of Sci-Kit Learn is different from what we may expect. 

In [ ]:
#Print model parameters
print('Intercept: \n', model1.intercept_)
print('Coefficients: \n', model1.coef_)

The first array contains three intercepts and the second  array contains three sets of regression coefficients.  
This is different from what we may be used to in SAS and R.  
In fact, the sklearn based output is different from the statsmodel version 
(A discussion of Multinomial Logistic Regression with statsmodels is available below). 


In this solution, there is an equation for each class. 
These act as independent binary logistic regression models. 
The actual output is log(p(y=c)/1 - p(y=c)), which are multinomial logit coefficients, 
hence the three equations.  After exponentiating each regressor coefficient, we in fact get odds ratios. 
The interpretation of the coefficients is for a single unit change in the predictor variable, 
the log of odds will change by a factor indicated by the beta coefficient, given that all other 
variables are held constant.  Log of odds is not really meaningful, so exponentiating the output gets a 
slightly more user friendly output: 

In [ ]:
#Calculate odds ratio estimates
import numpy as np
np.exp(model1.coef_)

Statsmodels:
Notice that the statsmodels output is very different from that of sklearn.  
In this case, there are K-1, in this case two equations, which show coefficients against a reference group. 
In the abalone example, the reference group was chosen to be female. 
The coefficients represent the log of ratios between two probabilities: 
the probability of belonging to a group of interest vs. the probability of belonging to the reference group.  
In the abalone example, the reference group was female, therefore the equation below represents the first set of 
coefficients marked as SEX=Infant.  
Note that there are two sets of coefficients, one marked as Infant and  the second marked as Male

In [ ]:
#Use statsmodels to assess variables

logit_model=sm.MNLogit(y_train,sm.add_constant(X_train))
logit_model
result=logit_model.fit()
stats1=result.summary()
stats2=result.summary2()
print(stats1)
print(stats2)

Accuracy:
Assessing the accuracy of the model is not difficult but errors at the different levels act as a compounding problem. 

In [ ]:
# scikit learn accuracy

In [ ]:
#Create a confusion matrix y_test as first argument and the preds as second argument 
confusion_matrix(y_test, preds)

#transform confusion matrix into array the matrix is stored in a vaiable called confmtrx
confmtrx = np.array(confusion_matrix(y_test, preds))

# Create DataFrame from confmtrx array  rows for test: Male, Female, Infant designation as index 
# columns for preds: male, predicted_female, predicted_infant as column

pd.DataFrame(confmtrx, index=['Female','Infant', 'Male'],
columns=['predicted_Female', 'predicted_Infant', 'predicted_Male'])

In [ ]:
#Accuracy statistics

print('Accuracy Score:', metrics.accuracy_score(y_test, preds))  

#Create classification report
class_report=classification_report(y_test, preds)
print(class_report)

The accuracy of this model is poor with only 55% of predictions being correct. 
The precision and recall of female and male abalone is very concerning as well.     

Task1: Load the iris dataset by using seaborn library and fit multinomial logistic regression  model.

# ordinal logistic regression

In [ ]:
import pandas as pd
data_diam = pd.read_csv('diamonds.csv')
data_diam.head(5)

In [ ]:
data_diam['cut'].value_counts()

Data preprocessing 
Here we can see that we have three variables in the object form and in this article we are dealing with the cut variable. 
To work with the ordinal models from statsmodel we are required to convert this target variable into a categorical ordered form that can be done using the following lines of codes:

In [ ]:
from pandas.api.types import CategoricalDtype
cat_type = CategoricalDtype(categories=['Fair', 'Good', 'Ideal', 'Very Good', 'Premium'], ordered=True)

data_diam["cut"] = data_diam["cut"].astype(cat_type)

data_diam['cut'].dtype

In [ ]:
# Now in the data, we have variables X, Y, and Z that represent the height, width, and depth of the diamond. 
# By multiplying them we can calculate the volume of the diamonds. Let’s calculate the volume.

data_diam['volume'] = data_diam['x'] * data_diam['y'] * data_diam['z']

data_diam.drop(['x','y','z'],axis=1,inplace=True)

In [ ]:
#Here we have multiplied the columns X, Y, and Z and dropped them from the data.
# Let’s plot the data to know about the distribution.

import matplotlib.pyplot as plt
 
plt.figure(figsize=[24,24])
 
plt.subplot(221)
plt.hist(data_diam['carat'],bins=20,color='b')
plt.xlabel('Weight')
plt.title('Distribution by Weight')
 
plt.subplot(222)
plt.hist(data_diam['depth'],bins=20,color='r')
plt.xlabel('Diamond Depth')
plt.title('Distribution by Depth')
 
plt.subplot(223)
plt.hist(data_diam['price'],bins=20,color='g')
plt.xlabel('Price')
plt.title('Distribution by Price')
 
plt.subplot(224)
plt.hist(data_diam['volume'],bins=20,color='m')
plt.xlabel('Volume')
plt.title('Distribution by Volume')

In [ ]:
# Ordered probit model
from statsmodels.miscmodels.ordinal_model import OrderedModel
mod_prob = OrderedModel(data_diam['cut'],
                        data_diam[['volume', 'price', 'carat']],
                        distr='probit')

In [ ]:
# In the above lines of codes, we have called the OrderedModel module that holds the function for the 
# ordinal regression and instantiates an Ordered probit model while taking the cut variable as our target and 
# volume, price, and carat as independent variables.
# We can fit and check the summary of the model using the following lines of codes:

res_prob = mod_prob.fit(method='bfgs')
res_prob.summary()

In [ ]:
# Ordered logit regression 
# Codes for this model are also similar to the above codes except for one thing we need to change is the parameter distr. In the above, we can see it is set as probit and needs to change in logit. 

mod_prob = OrderedModel(data_diam['cut'],
                        data_diam[['volume', 'price', 'carat']],
                        distr='logit')
 
res_log = mod_prob.fit(method='bfgs')
res_log.summary()

In [ ]:
#Now we can make the prediction from the model.

predicted = res_log.model.predict(res_log.params, exog=data_diam[['volume', 'price', 'carat']])
predicted